In [2]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 
import astropy

#latlon = pd.read_csv('NewLempo_LatLon.csv', header=0)

latlon = pd.read_csv('Lempo_obs_df_old.csv', header=0)

date = latlon['time']
dateList = []
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
        
primary = Horizons(id='Lempo',location=None,epochs=dateList)
    
dist = primary.vectors()['range']

Lat_Prim = np.array(latlon['Lat_Prim'])
Long_Prim = np.array(latlon['Long_Prim'])
DLat_Paha = np.array(latlon['DeltaLat_Paha'])
DLong_Paha = np.array(latlon['DeltaLong_Paha'])
DLat_Hiisi = np.array(latlon['DeltaLat_Hiisi'])
DLong_Hiisi = np.array(latlon['DeltaLong_Hiisi'])

Lat2 = Lat_Prim + DLat_Paha/3600
Long2 = DLong_Paha/3600/np.cos(Lat_Prim*u.degree)+Long_Prim
Lat3 = Lat_Prim + DLat_Hiisi/3600
Long3 = DLong_Hiisi/3600/np.cos(Lat_Prim*u.degree)+Long_Prim

primEcl = SkyCoord(lat=np.array(Lat_Prim)*u.degree, lon=np.array(Long_Prim)*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

PahaEcl = SkyCoord(lat=np.array(Lat2)*u.degree, lon=np.array(Long2)*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))
HiisiEcl = SkyCoord(lat=np.array(Lat3)*u.degree, lon=np.array(Long3)*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

Paha = PahaEcl.transform_to('icrs')
Hiisi = HiisiEcl.transform_to('icrs')
prim = primEcl.transform_to('icrs')

RA1 = prim.ra.degree
DEC1 = prim.dec.degree
RA2 = Paha.ra.degree
DEC2 = Paha.dec.degree
RA3 = Hiisi.ra.degree
DEC3 = Hiisi.dec.degree

#print(RA2)

newdf = pd.DataFrame()
newdf['time'] = date
newdf['RA_Prim'] = RA1
newdf['DEC_Prim'] = DEC1
newdf['DRA2'] = (RA2-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC2'] = (DEC2-DEC1)*3600
newdf['DRA3'] = (RA3-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC3'] = (DEC3-DEC1)*3600

print(newdf)


          time    RA_Prim  DEC_Prim      DRA2   DDEC2      DRA3   DDEC3
0  2452252.405   4.228939 -7.289661  0.003524  0.3812 -0.045809  0.0367
1  2452252.806   4.231229 -7.288711  0.071295  0.3750  0.059999  0.0244
2  2452820.571   7.474369 -5.925691  0.288275  0.1839 -0.025271 -0.0084
3  2452830.309   7.530149 -5.902001  0.244114 -0.1081 -0.022504  0.0154
4  2452846.247   7.621469 -5.863211 -0.171696 -0.0296  0.025711  0.0040
5  2452875.057   7.786559 -5.793031  0.354763  0.0447  0.025711 -0.0214
6  2453152.263   9.377529 -5.113491 -0.365058  0.1474 -0.048981 -0.0241
7  2453595.477  11.930229 -4.012251  0.038550 -0.2734 -0.034525  0.0167
8  2453923.040  13.823839 -3.188101 -0.258179  0.2702  0.017392  0.0258


In [56]:
radec = pd.read_csv('BenecchiData_NotWrong.csv', header=0)

#latlon = pd.read_csv('Lempo_obs_df_old.csv', header=0)

latlon = pd.DataFrame(columns = ['time'])
    
date = radec['time']
dateList = []
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
        
    #Get the Horizons data for the object at the times it was observed
primary = Horizons(id='Lempo',location=None,epochs=dateList)
    
latlon['time'] = radec['time']-primary.vectors()['lighttime']
    
    #Pull all data from csv file
RA_Prim = np.array(radec['RA-Primary'])
DEC_Prim = np.array(radec['DEC-Primary'])
    
deltaRA_Paha = np.array(radec['Delta-RA_Paha'])
deltaDEC_Paha = np.array(radec['Delta-DEC_Paha'])
deltaRA_Hiisi = np.array(radec['Delta-RA_Hiisi'])
deltaDEC_Hiisi = np.array(radec['Delta-DEC_Hiisi'])
    
    #Convert the deltas back to degrees from arcseconds

RA_Paha = RA_Prim+deltaRA_Paha/3600/np.cos(DEC_Prim*u.degree)
DEC_Paha = DEC_Prim + deltaDEC_Paha/3600
      
RA_Hiisi = RA_Prim+deltaRA_Hiisi/3600/np.cos(DEC_Prim*u.degree)
DEC_Hiisi = DEC_Prim + deltaDEC_Hiisi/3600
  
radf = pd.DataFrame()
radf['RA-Prim'] = RA_Prim
radf['DEC-Prim'] = DEC_Prim

radf['DRA_Paha'] = deltaRA_Paha
radf['DDEC_Paha'] = deltaDEC_Paha
radf['DRA_Hiisi'] = deltaRA_Hiisi
radf['DDEC_Hiisi'] = deltaDEC_Hiisi
#print(radf)
    
    #Essentially we define where the object is in our RA/DEC coordinate system. ICRS is the system our coordinates are in.
dist = primary.vectors()['range']
PahaC = SkyCoord(ra=RA_Paha*u.degree, dec=DEC_Paha*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
HiisiC = SkyCoord(ra=RA_Hiisi*u.degree, dec=DEC_Hiisi*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
        
primC = SkyCoord(ra=RA_Prim*u.degree, dec=DEC_Prim*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    #Transform your icrs frame to a J2000 latitude and longitude ecliptic frame
PahaEcl = PahaC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
HiisiEcl = HiisiC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
primEcl = primC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
Lat_Prim = primEcl.lat.degree
Long_Prim = primEcl.lon.degree
    
Lat_Paha = PahaEcl.lat.degree
Long_Paha = PahaEcl.lon.degree
    
Lat_Hiisi = HiisiEcl.lat.degree
Long_Hiisi = HiisiEcl.lon.degree

DeltaLat_Paha = (Lat_Paha-Lat_Prim)*3600
DeltaLong_Paha = (Long_Paha-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
DeltaLat_Hiisi = (Lat_Hiisi-Lat_Prim)*3600
DeltaLong_Hiisi = (Long_Hiisi-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600

Lat_Paha = Lat_Prim + DeltaLat_Paha/3600
Long_Paha = DeltaLong_Paha/3600/np.cos(Lat_Prim*u.degree)+Long_Prim
Lat_Hiisi = Lat_Prim + DeltaLat_Hiisi/3600
Long_Hiisi = DeltaLong_Hiisi/3600/np.cos(Lat_Prim*u.degree)+Long_Prim


print(Lat_Prim)
print(np.cos(Lat_Prim*u.degree))
print(DeltaLong_Paha/3600/np.cos(Lat_Prim*u.degree))

latlon['Lat_Prim'] = Lat_Prim
latlon['Long_Prim'] = Long_Prim
latlon['DLat_Paha'] = DeltaLat_Paha
latlon['DLong_Paha'] = DeltaLong_Paha
latlon['DLat_Hiisi'] = DeltaLat_Hiisi
latlon['DLong_Hiisi'] = DeltaLong_Hiisi

#print(Long_Paha)
#print(latlon)

primEcl = SkyCoord(lat=Lat_Prim*u.degree, lon=Long_Prim*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

PahaEcl = SkyCoord(lat=Lat_Paha*u.degree, lon=Long_Paha*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))
HiisiEcl = SkyCoord(lat=Lat_Hiisi*u.degree, lon=Long_Hiisi*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

Paha = PahaEcl.transform_to('icrs')
Hiisi = HiisiEcl.transform_to('icrs')
prim = primEcl.transform_to('icrs')

RA1 = prim.ra.degree
DEC1 = prim.dec.degree
RA2 = Paha.ra.degree
DEC2 = Paha.dec.degree
RA3 = Hiisi.ra.degree
DEC3 = Hiisi.dec.degree

#print(RA2)
newdf = pd.DataFrame()
newdf['time'] = date
newdf['RA_Prim'] = RA1
newdf['DEC_Prim'] = DEC1
newdf['DRA2'] = (RA2-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC2'] = (DEC2-DEC1)*3600
newdf['DRA3'] = (RA3-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC3'] = (DEC3-DEC1)*3600


print(newdf)

[-8.42533684 -8.42174164 -8.41638759 -8.46083859 -8.53084126 -8.63428733
 -8.2728092  -8.58773098 -8.39879024]
[0.98920764 0.98921683 0.98923051 0.98911666 0.98893616 0.98866672
 0.98959418 0.98878838 0.98927542]
[ 4.34762460e-05  5.21861703e-05  9.04504551e-05  4.66581915e-05
 -4.39197659e-05  8.60795243e-05 -2.05615817e-05 -3.64927095e-05
  9.62301539e-05]
           time    RA_Prim  DEC_Prim    DRA2   DDEC2    DRA3   DDEC3
0  2.452252e+06   2.639207 -8.045529  0.0019  0.3812 -0.0247  0.0367
1  2.452253e+06   2.637457 -8.042362  0.0385  0.3750  0.0324  0.0244
2  2.452821e+06   9.206541 -5.191167  0.2715  0.1839 -0.0238 -0.0084
3  2.452830e+06   9.250458 -5.220695  0.2278 -0.1081 -0.0210  0.0154
4  2.452846e+06   9.224166 -5.308306 -0.1576 -0.0296  0.0236  0.0040
5  2.452875e+06   8.890458 -5.564862  0.3146  0.0447  0.0228 -0.0214
6  2.453152e+06  10.777916 -4.359645 -0.1431  0.1474 -0.0192 -0.0241
7  2.453595e+06  13.334125 -3.611695 -0.0249 -0.2734  0.0223  0.0167
8  2.453923e+06  1